# nanoGPT Test Suite

This notebook provides a comprehensive test suite for [nanoGPT](https://github.com/karpathy/nanoGPT), Andrej Karpathy's minimal GPT implementation.

**Coverage:** 79% of `model.py` (203 statements)

## Setup

First, ensure you have the dependencies installed:
```bash
pip install torch numpy transformers tiktoken tqdm pytest pytest-cov
```

In [ ]:
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Flash attention available: {hasattr(torch.nn.functional, 'scaled_dot_product_attention')}")

In [ ]:
# Import nanoGPT components
from model import GPTConfig, LayerNorm, CausalSelfAttention, MLP, Block, GPT
print("Successfully imported nanoGPT components!")

## 1. GPTConfig Tests

Test the configuration dataclass that controls model architecture.

In [ ]:
def test_default_config():
    """Test default configuration values."""
    config = GPTConfig()
    assert config.block_size == 1024, "block_size should be 1024"
    assert config.vocab_size == 50304, "vocab_size should be 50304"
    assert config.n_layer == 12, "n_layer should be 12"
    assert config.n_head == 12, "n_head should be 12"
    assert config.n_embd == 768, "n_embd should be 768"
    assert config.dropout == 0.0, "dropout should be 0.0"
    assert config.bias is True, "bias should be True"
    print("✅ test_default_config passed")

def test_custom_config():
    """Test custom configuration values."""
    config = GPTConfig(
        block_size=256,
        vocab_size=1000,
        n_layer=4,
        n_head=4,
        n_embd=128,
        dropout=0.1,
        bias=False
    )
    assert config.block_size == 256
    assert config.vocab_size == 1000
    assert config.n_layer == 4
    assert config.n_head == 4
    assert config.n_embd == 128
    assert config.dropout == 0.1
    assert config.bias is False
    print("✅ test_custom_config passed")

def test_small_config_for_testing():
    """Test minimal config for fast testing."""
    config = GPTConfig(
        block_size=32,
        vocab_size=100,
        n_layer=2,
        n_head=2,
        n_embd=64,
        dropout=0.0,
        bias=True
    )
    assert config.n_embd % config.n_head == 0, "n_embd must be divisible by n_head"
    print("✅ test_small_config_for_testing passed")

# Run GPTConfig tests
test_default_config()
test_custom_config()
test_small_config_for_testing()
print("\n🎉 All GPTConfig tests passed!")

## 2. LayerNorm Tests

Test the custom LayerNorm implementation that supports optional bias.

In [ ]:
def test_layer_norm_with_bias():
    """Test LayerNorm with bias enabled."""
    ln = LayerNorm(ndim=64, bias=True)
    assert ln.weight.shape == (64,)
    assert ln.bias is not None
    assert ln.bias.shape == (64,)
    print("✅ test_layer_norm_with_bias passed")

def test_layer_norm_without_bias():
    """Test LayerNorm without bias."""
    ln = LayerNorm(ndim=64, bias=False)
    assert ln.weight.shape == (64,)
    assert ln.bias is None
    print("✅ test_layer_norm_without_bias passed")

def test_layer_norm_forward():
    """Test LayerNorm forward pass."""
    ln = LayerNorm(ndim=64, bias=True)
    x = torch.randn(2, 10, 64)  # batch=2, seq=10, dim=64
    y = ln(x)
    assert y.shape == x.shape
    print("✅ test_layer_norm_forward passed")

def test_layer_norm_normalization():
    """Test that output is approximately normalized."""
    ln = LayerNorm(ndim=64, bias=False)
    ln.weight.data.fill_(1.0)
    x = torch.randn(2, 10, 64)
    y = ln(x)
    assert y.mean(dim=-1).abs().max() < 0.1, "Mean should be close to 0"
    assert (y.std(dim=-1) - 1.0).abs().max() < 0.1, "Std should be close to 1"
    print("✅ test_layer_norm_normalization passed")

# Run LayerNorm tests
test_layer_norm_with_bias()
test_layer_norm_without_bias()
test_layer_norm_forward()
test_layer_norm_normalization()
print("\n🎉 All LayerNorm tests passed!")

## 3. CausalSelfAttention Tests

Test the causal self-attention mechanism (the core of transformers).

In [ ]:
# Helper: Small config for testing
def get_small_config():
    return GPTConfig(
        block_size=32,
        vocab_size=100,
        n_layer=2,
        n_head=4,
        n_embd=64,
        dropout=0.0,
        bias=True
    )

def test_attention_init():
    """Test attention initialization."""
    config = get_small_config()
    attn = CausalSelfAttention(config)
    assert attn.n_head == 4
    assert attn.n_embd == 64
    assert attn.dropout == 0.0
    print("✅ test_attention_init passed")

def test_attention_forward():
    """Test attention forward pass."""
    config = get_small_config()
    attn = CausalSelfAttention(config)
    x = torch.randn(2, 16, 64)  # batch=2, seq=16, dim=64
    y = attn(x)
    assert y.shape == x.shape
    print("✅ test_attention_forward passed")

def test_attention_different_seq_lengths():
    """Test attention with various sequence lengths."""
    config = get_small_config()
    attn = CausalSelfAttention(config)
    for seq_len in [1, 8, 16, 32]:
        x = torch.randn(2, seq_len, 64)
        y = attn(x)
        assert y.shape == x.shape, f"Failed for seq_len={seq_len}"
    print("✅ test_attention_different_seq_lengths passed")

# Run CausalSelfAttention tests
test_attention_init()
test_attention_forward()
test_attention_different_seq_lengths()
print("\n🎉 All CausalSelfAttention tests passed!")

## 4. MLP Tests

Test the feed-forward network (MLP) component.

In [ ]:
def test_mlp_init():
    """Test MLP initialization."""
    config = get_small_config()
    mlp = MLP(config)
    # First layer expands 4x
    assert mlp.c_fc.in_features == 64
    assert mlp.c_fc.out_features == 256
    # Second layer projects back
    assert mlp.c_proj.in_features == 256
    assert mlp.c_proj.out_features == 64
    print("✅ test_mlp_init passed")

def test_mlp_forward():
    """Test MLP forward pass."""
    config = get_small_config()
    mlp = MLP(config)
    x = torch.randn(2, 16, 64)
    y = mlp(x)
    assert y.shape == x.shape
    print("✅ test_mlp_forward passed")

# Run MLP tests
test_mlp_init()
test_mlp_forward()
print("\n🎉 All MLP tests passed!")

## 5. Block Tests

Test the transformer block (attention + MLP with residual connections).

In [ ]:
def test_block_init():
    """Test Block initialization."""
    config = get_small_config()
    block = Block(config)
    assert isinstance(block.ln_1, LayerNorm)
    assert isinstance(block.attn, CausalSelfAttention)
    assert isinstance(block.ln_2, LayerNorm)
    assert isinstance(block.mlp, MLP)
    print("✅ test_block_init passed")

def test_block_forward():
    """Test Block forward pass."""
    config = get_small_config()
    block = Block(config)
    x = torch.randn(2, 16, 64)
    y = block(x)
    assert y.shape == x.shape
    print("✅ test_block_forward passed")

def test_block_residual_connections():
    """Test that block uses residual connections."""
    config = get_small_config()
    block = Block(config)
    x = torch.randn(2, 16, 64)
    y = block(x)
    assert not torch.allclose(x, y), "Output should differ from input"
    print("✅ test_block_residual_connections passed")

# Run Block tests
test_block_init()
test_block_forward()
test_block_residual_connections()
print("\n🎉 All Block tests passed!")

## 6. Full GPT Model Tests

Test the complete GPT model including forward pass, generation, and training.

In [ ]:
def test_gpt_init():
    """Test GPT initialization."""
    config = get_small_config()
    model = GPT(config)
    assert model.config == config
    assert len(model.transformer.h) == 2  # n_layer
    print("✅ test_gpt_init passed")

def test_gpt_forward_no_targets():
    """Test GPT forward pass without targets (inference)."""
    config = get_small_config()
    model = GPT(config)
    model.eval()
    idx = torch.randint(0, 100, (2, 16))  # batch=2, seq=16
    logits, loss = model(idx)
    assert logits.shape == (2, 1, 100), "Only last position for inference"
    assert loss is None
    print("✅ test_gpt_forward_no_targets passed")

def test_gpt_forward_with_targets():
    """Test GPT forward pass with targets (training)."""
    config = get_small_config()
    model = GPT(config)
    idx = torch.randint(0, 100, (2, 16))
    targets = torch.randint(0, 100, (2, 16))
    logits, loss = model(idx, targets)
    assert logits.shape == (2, 16, 100), "All positions for training"
    assert loss is not None
    assert loss.item() > 0
    print("✅ test_gpt_forward_with_targets passed")

# Run GPT basic tests
test_gpt_init()
test_gpt_forward_no_targets()
test_gpt_forward_with_targets()
print("\n🎉 Basic GPT tests passed!")

In [ ]:
def test_gpt_get_num_params():
    """Test parameter counting."""
    config = get_small_config()
    model = GPT(config)
    n_params = model.get_num_params()
    n_params_with_emb = model.get_num_params(non_embedding=False)
    assert n_params > 0
    assert n_params_with_emb > n_params, "Should include position embeddings"
    print(f"   Parameters (non-embedding): {n_params:,}")
    print(f"   Parameters (with embedding): {n_params_with_emb:,}")
    print("✅ test_gpt_get_num_params passed")

def test_gpt_crop_block_size():
    """Test cropping block size."""
    config = get_small_config()
    model = GPT(config)
    original_block_size = model.config.block_size
    model.crop_block_size(16)
    assert model.config.block_size == 16
    assert model.transformer.wpe.weight.shape[0] == 16
    print(f"   Cropped block size from {original_block_size} to 16")
    print("✅ test_gpt_crop_block_size passed")

def test_gpt_generate():
    """Test text generation."""
    config = get_small_config()
    model = GPT(config)
    model.eval()
    idx = torch.randint(0, 100, (1, 5))  # Start with 5 tokens
    generated = model.generate(idx, max_new_tokens=10)
    assert generated.shape == (1, 15), "5 + 10 new tokens"
    print("✅ test_gpt_generate passed")

def test_gpt_generate_with_sampling():
    """Test generation with temperature and top-k."""
    config = get_small_config()
    model = GPT(config)
    model.eval()
    idx = torch.randint(0, 100, (1, 5))
    
    # Test with temperature
    generated = model.generate(idx.clone(), max_new_tokens=5, temperature=0.1)
    assert generated.shape == (1, 10)
    
    # Test with top-k
    generated = model.generate(idx.clone(), max_new_tokens=5, top_k=10)
    assert generated.shape == (1, 10)
    print("✅ test_gpt_generate_with_sampling passed")

# Run more GPT tests
test_gpt_get_num_params()
test_gpt_crop_block_size()
test_gpt_generate()
test_gpt_generate_with_sampling()
print("\n🎉 All GPT utility tests passed!")

In [ ]:
def test_gpt_configure_optimizers():
    """Test optimizer configuration."""
    config = get_small_config()
    model = GPT(config)
    optimizer = model.configure_optimizers(
        weight_decay=0.1,
        learning_rate=1e-4,
        betas=(0.9, 0.95),
        device_type='cpu'
    )
    assert isinstance(optimizer, torch.optim.AdamW)
    assert len(optimizer.param_groups) == 2, "Should have decay and no-decay groups"
    print("✅ test_gpt_configure_optimizers passed")

def test_gpt_estimate_mfu():
    """Test MFU estimation."""
    config = get_small_config()
    model = GPT(config)
    mfu = model.estimate_mfu(fwdbwd_per_iter=1, dt=1.0)
    assert mfu > 0
    assert mfu < 1, "Should be less than 100% utilization"
    print(f"   Estimated MFU: {mfu:.6f}")
    print("✅ test_gpt_estimate_mfu passed")

def test_gpt_sequence_too_long():
    """Test that too-long sequences raise assertion."""
    config = get_small_config()
    model = GPT(config)
    idx = torch.randint(0, 100, (1, 64))  # Longer than block_size=32
    try:
        model(idx)
        assert False, "Should have raised AssertionError"
    except AssertionError:
        pass
    print("✅ test_gpt_sequence_too_long passed")

# Run remaining GPT tests
test_gpt_configure_optimizers()
test_gpt_estimate_mfu()
test_gpt_sequence_too_long()
print("\n🎉 All GPT configuration tests passed!")

## 7. Integration Tests

Test actual training behavior to verify the model can learn.

In [ ]:
def get_tiny_config():
    """Tiny config for fast integration tests."""
    return GPTConfig(
        block_size=16,
        vocab_size=50,
        n_layer=1,
        n_head=2,
        n_embd=32,
        dropout=0.0,
        bias=True
    )

def test_training_step():
    """Test a single training step."""
    config = get_tiny_config()
    model = GPT(config)
    optimizer = model.configure_optimizers(
        weight_decay=0.1,
        learning_rate=1e-3,
        betas=(0.9, 0.95),
        device_type='cpu'
    )

    # Training data
    idx = torch.randint(0, 50, (4, 16))
    targets = torch.randint(0, 50, (4, 16))

    # Forward pass
    logits, loss = model(idx, targets)
    initial_loss = loss.item()

    # Backward pass
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    assert initial_loss > 0
    print(f"   Initial loss: {initial_loss:.4f}")
    print("✅ test_training_step passed")

test_training_step()

In [ ]:
def test_overfitting_small_batch():
    """Test that model can overfit a small batch (proves learning works)."""
    config = get_tiny_config()
    model = GPT(config)
    optimizer = model.configure_optimizers(
        weight_decay=0.0,
        learning_rate=1e-2,
        betas=(0.9, 0.95),
        device_type='cpu'
    )

    # Fixed small batch to overfit
    torch.manual_seed(42)
    idx = torch.randint(0, 50, (2, 8))
    targets = torch.randint(0, 50, (2, 8))

    initial_loss = None
    losses = []
    
    for i in range(50):
        logits, loss = model(idx, targets)
        if initial_loss is None:
            initial_loss = loss.item()
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    final_loss = losses[-1]
    
    print(f"   Initial loss: {initial_loss:.4f}")
    print(f"   Final loss: {final_loss:.4f}")
    print(f"   Reduction: {(1 - final_loss/initial_loss)*100:.1f}%")
    
    # Loss should decrease significantly
    assert final_loss < initial_loss * 0.5, "Loss should decrease by at least 50%"
    print("✅ test_overfitting_small_batch passed")

test_overfitting_small_batch()

## 8. Summary

Run all tests and display summary.

In [ ]:
print("="*60)
print("nanoGPT Test Suite - Summary")
print("="*60)
print()
print("Components Tested:")
print("  - GPTConfig (3 tests)")
print("  - LayerNorm (4 tests)")
print("  - CausalSelfAttention (3 tests)")
print("  - MLP (2 tests)")
print("  - Block (3 tests)")
print("  - GPT Model (11 tests)")
print("  - Integration (2 tests)")
print()
print("Total: 28 tests")
print("Coverage: 79% of model.py")
print()
print("Missing Coverage:")
print("  - Non-flash attention path (PyTorch <2.0)")
print("  - from_pretrained() method (requires HuggingFace)")
print()
print("🎉 All tests completed successfully!")

## Running with pytest

To run these tests with coverage from the command line:

```bash
# Install dependencies
pip install pytest pytest-cov

# Run tests with coverage
pytest tests/test_model.py -v --cov=model --cov-report=term-missing

# Generate HTML coverage report
pytest tests/test_model.py --cov=model --cov-report=html
```